In [1]:
import os
os.chdir('..')
from inputGetter import get_input

content = get_input(2024, 17)[:-1]

# Part A
From different input sets (that I obtained through my different email accounts), it's noticed that the programs in the input sets, while might be a little bit different, all share some similarities:

- The instruction 3, which is to jump to a specific position program if register A's value > 0, is only used once in the end of a program (3, 0). Basically this just jumps back to first instruction of the program again, make this essentially a while loop with the condition of the value for register A
- Value of register A is only changed...once in the program. With instruction (0, 3) specifically. This means that, register A's value will lose 3 least significant bits each time, and that also means that, if output has n numbers, there are 3 * n bits in this number (including possibly 1 or 2 leading zeros).
- Instruction for output is only used once with (5, 5). This means that for each time, it will output the 3 least significant bits in the register B's value. Therefore, this is good for us, by just tracking each block of 3 bits, starting from least, then to most significant 3-bits group for register A's value. We can get it. Besides that, it can be seen that register B and C's values are reset and dependent in register A's value in each iteration, makes it easier for us.

These are some of very important observations, as it will help us control the code for this in specific way. While I believe that this pattern would appear in most, if not all of the inputs for all accounts, I could be wrong. So if this is the case, this code would be a generic guide for it.

In [2]:
content = content.split('\n')

register_a, register_b, register_c = [int(line.split()[-1]) for line in content[0:3]]
program = content[-1].split()[-1]
program = [int(val) for val in program.split(',')]

# This was the code used to get the steps for this
print("Steps")
for i in range(0, len(program), 2):
    opcode, literal = program[i: i+2]
    
    comb = literal
    if comb == 4:
        comb = 'register_a'
    elif comb == 5:
        comb = 'register_b'
    elif comb == 6:
        comb = 'register_c'

    print('- ', end='')
        
    if opcode == 0:
        print(f'register_a = register_a >> {comb}')
    elif opcode == 1:
        print(f'register_b = register_b ^ {literal}')
    elif opcode == 2:
        print(f'register_b = {comb} % 8')
    elif opcode == 3:
        print(f'Loop back if register_a > 0')
    elif opcode == 4:
        print(f'register_b = register_b ^ register_c')
    elif opcode == 5:
        print(f'Output the {comb} % 8')
    elif opcode == 6:
        print(f'register_b = register_a >> {comb}')
    else:
        print(f'register_c = register_a >> {comb}')

Steps
- register_b = register_a % 8
- register_b = register_b ^ 5
- register_c = register_a >> register_b
- register_b = register_b ^ 6
- register_b = register_b ^ register_c
- Output the register_b % 8
- register_a = register_a >> 3
- Loop back if register_a > 0


In [3]:
'''Process all the instructions, except jump, because that will be 
handled for part A and B differently, so can't be shared'''
def process_instruction(opcode, literal):
    global register_a, register_b, register_c
    comb = literal
    if comb == 4:
        comb = register_a
    elif comb == 5:
        comb = register_b
    elif comb == 6:
        comb = register_c
    
    if opcode == 0:
        register_a //= 2 ** comb
    elif opcode == 1:
        register_b ^= literal
    elif opcode == 2:
        register_b = comb % 8
    elif opcode == 4:
        register_b ^= register_c
    elif opcode == 5:
        return comb % 8
    elif opcode == 6:
        register_b = register_a // (1 << comb)
    elif opcode == 7:
        register_c = register_a // (1 << comb)

In [4]:
inst_id = 0
res_a = []
while inst_id < len(program):
    opcode, literal = program[inst_id: inst_id+2]
    if opcode == 3 and register_a != 0:
        inst_id = literal
        continue
    
    if opcode == 5:
        res_a.append(process_instruction(opcode, literal))
    else:
        process_instruction(opcode, literal)
    inst_id += 2

print(f"Part A: {','.join([str(val) for val in res_a])}")

Part A: 3,5,0,1,5,1,5,1,0


# Part B

Extra observations from this: Value of register A is only used at the beginning, and not affected, only got shifted right by 3 bits in the end, so the change of register A's value is independent from the change of register B and C's value. Because of this, for backtracking, I will skip instruction (0, 3), (5, 5) and (3, 0) and only focus on testing value of 3 least significant bits each time (because these instructions don't quite help on shaping the result), see if it can produce the right output

In [5]:
res_b = 100000000000000000000
def backtrack(program_id, curr_num):
    global res_b, register_a, register_b, register_c
    if program_id < 0:
        res_b = min(res_b, curr_num)
        return
    
    # Go through value 0-7, and test if this would make the desired output
    for i in range(8):
        register_a = (curr_num << 3) + i
        for inst_id in range(0, len(program), 2):
            opcode, literal = program[inst_id: inst_id+2]
            if opcode != 0:
                # Just backtrack in right away, because in the input, no change will be made (or mattered) after this one
                output = process_instruction(opcode, literal)
                if opcode == 5 and output == program[program_id]:
                    backtrack(program_id-1, register_a)

backtrack(len(program) - 1, 0)
print(f'Part B: {res_b}')

Part B: 107413700225434
